# Scrape US Treasury Rates and plot the Yield Curve

Import packages

In [1]:
import pandas as pd
import numpy as np
import requests
import xmltodict
import plotly.graph_objects as go

Fetch and parse the Data

In [2]:
url="https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData"
r=requests.get(url)

In [3]:
content_dict=xmltodict.parse(r.content)

Consolidate into a single dataframe

In [4]:
yield_data_list=[]
for i in range(len(content_dict['feed']['entry'])):
    yield_data_list.append(pd.DataFrame(content_dict['feed']['entry'][i]['content']['m:properties']))

In [5]:
yield_data=pd.concat(yield_data_list, ignore_index=True)

Some data processing

In [6]:
yield_data.columns=[c.strip('d:') for c in yield_data.columns]

In [7]:
yield_data['I']=pd.to_numeric(yield_data['I'], errors='coerce')
yield_data=yield_data[~yield_data['I'].isna()]

In [8]:
yield_data['NEW_DATE']=yield_data['NEW_DATE'].apply(lambda x: pd.to_datetime(x))

In [9]:
data=pd.melt(yield_data, id_vars=['I','NEW_DATE'], value_vars=['BC_1MONTH','BC_3MONTH', 'BC_6MONTH','BC_1YEAR','BC_2YEAR','BC_3YEAR','BC_5YEAR','BC_7YEAR','BC_10YEAR','BC_20YEAR','BC_30YEAR'])

In [10]:
data['variable_'],uniques=pd.factorize(data['variable'])

In [11]:
data['value']=pd.to_numeric(data['value'], errors='coerce')

In [12]:
data=data[data['NEW_DATE']!='2017-04-14'] 

Plot the data

In [13]:


fig = go.Figure(data=[go.Mesh3d(z=data.value.values, x=data.NEW_DATE.values, y=data.variable.values, opacity=1, intensity=data.value.values, colorscale='RdBu',cmin=-2,cmax=6)])


fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=8, range=[np.datetime64('2001-01-01'),np.datetime64('2021-01-01')],),
        yaxis = dict(nticks=10, range=[1,10],),
        zaxis = dict(nticks=13, range=[-2,8],),),
    width=800,
    height=800,
    margin=dict(r=20, l=10, b=10, t=10))

#fig.show()
fig.write_image("fig1.png")